In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy import stats

In [2]:
url = "../data/Train.csv"
df = pd.read_csv(url)
df.head()


,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
df["Ever_Married"].unique()

array(['No', 'Yes', nan], dtype=object)

In [4]:
df.shape

(8068, 11)

In [5]:
df = df.drop(columns = ['Var_1', 'Segmentation', "ID"])

In [6]:
df.isnull().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
dtype: int64

In [7]:
df.dropna(inplace = True)
df.shape

(6718, 8)

In [8]:
df["Graduated"].unique()

array(['No', 'Yes'], dtype=object)

In [9]:
df.isnull().sum()

Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
dtype: int64

In [10]:
df.duplicated().sum()

1135

In [11]:
# One-hot encoding for sex and title
df_dummies = pd.get_dummies(df, columns=["Profession"])
df_dummies

,Gender,Ever_Married,Age,Graduated,Work_Experience,Spending_Score,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing
0,Male,No,22,No,1.0,Low,4.0,False,False,False,False,False,True,False,False,False
2,Female,Yes,67,Yes,1.0,Low,1.0,False,False,True,False,False,False,False,False,False
3,Male,Yes,67,Yes,0.0,High,2.0,False,False,False,False,False,False,False,True,False
5,Male,Yes,56,No,0.0,Average,2.0,True,False,False,False,False,False,False,False,False
6,Male,No,32,Yes,1.0,Low,3.0,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,Male,Yes,41,Yes,0.0,High,5.0,True,False,False,False,False,False,False,False,False
8064,Male,No,35,No,3.0,Low,4.0,False,False,False,False,True,False,False,False,False
8065,Female,No,33,Yes,1.0,Low,1.0,False,False,False,False,False,True,False,False,False
8066,Female,No,27,Yes,1.0,Low,4.0,False,False,False,False,False,True,False,False,False


In [12]:
gender_map = {"Female": True, "Male": False}
df_dummies["Gender"] = df_dummies["Gender"].map(gender_map)

In [13]:
def mapeo(df, column):

    map_total = {"Yes": True, "No": False}
    df[column] = df[column].map(map_total)
    return df

In [14]:
mapeo(df_dummies, "Ever_Married")
mapeo(df_dummies, "Graduated")

,Gender,Ever_Married,Age,Graduated,Work_Experience,Spending_Score,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,Profession_Marketing
0,False,False,22,False,1.0,Low,4.0,False,False,False,False,False,True,False,False,False
2,True,True,67,True,1.0,Low,1.0,False,False,True,False,False,False,False,False,False
3,False,True,67,True,0.0,High,2.0,False,False,False,False,False,False,False,True,False
5,False,True,56,False,0.0,Average,2.0,True,False,False,False,False,False,False,False,False
6,False,False,32,True,1.0,Low,3.0,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,False,True,41,True,0.0,High,5.0,True,False,False,False,False,False,False,False,False
8064,False,False,35,False,3.0,Low,4.0,False,False,False,False,True,False,False,False,False
8065,True,False,33,True,1.0,Low,1.0,False,False,False,False,False,True,False,False,False
8066,True,False,27,True,1.0,Low,4.0,False,False,False,False,False,True,False,False,False


In [15]:
features = df_dummies.drop('Spending_Score', axis=1)
target = df_dummies['Spending_Score']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
knn = KNeighborsClassifier(n_neighbors=8)

In [18]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=8)

In [19]:
pred = knn.predict(X_test) 
pred

array(['Low', 'High', 'Low', ..., 'Average', 'Low', 'Low'], dtype=object)

In [20]:
knn.score(X_test, y_test) 

0.7455357142857143

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy import stats

# Función para eliminar outliers
def remove_outliers(df, columns, z_threshold=3):
    columns = [col for col in columns if col in df.columns]
    return df[(np.abs(stats.zscore(df[columns])) < z_threshold).all(axis=1)]

# Definir diferentes conjuntos de características
all_features = features.columns.tolist()
numeric_features = [col for col in ['Age', 'Work_Experience', 'Family_Size'] if col in features.columns]
categorical_features = [col for col in ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score', 
                                        'Profession_Artist', 'Profession_Doctor', 'Profession_Engineer', 
                                        'Profession_Entertainment', 'Profession_Executive', 'Profession_Healthcare', 
                                        'Profession_Homemaker', 'Profession_Lawyer', 'Profession_Marketing'] if col in features.columns]

feature_sets = {
    'All Features': all_features,
    'Numeric Only': numeric_features,
    'Categorical Only': categorical_features,
    'No Family Size': [col for col in all_features if col != 'Family_Size']
}

# Definir modelos
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier()
}

# Preparar resultados en un DataFrame
results = []

# Iterar a través de diferentes escenarios
for outliers in ['With Outliers', 'Without Outliers']:
    for scaling in ['No Scaling', 'Standardization', 'Normalization']:
        for feature_set_name, features_list in feature_sets.items():
            # Preparar los datos
            X_subset = features[features_list]
            y_subset = target
            
            if outliers == 'Without Outliers':
                X_subset = remove_outliers(X_subset, numeric_features)
                y_subset = y_subset[X_subset.index]
            
            # Dividir los datos en train y test
            X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)
            
            # Aplicar escalado si es necesario
            if scaling == 'Standardization':
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
            elif scaling == 'Normalization':
                scaler = MinMaxScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)
            else:
                X_train_scaled = X_train
                X_test_scaled = X_test
            
            # Entrenar y evaluar modelos
            for model_name, model in models.items():
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
                accuracy = accuracy_score(y_test, y_pred)
                
                results.append({
                    'Model': model_name,
                    'Outliers': outliers,
                    'Scaling': scaling,
                    'Feature Set': feature_set_name,
                    'Accuracy': accuracy
                })

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results)

# Imprimir resumen
print(results_df)

# Encontrar el mejor modelo
best_model = results_df.loc[results_df['Accuracy'].idxmax()]
print("\nBest performing model:")
print(best_model)

# Opcional: Guardar los resultados en un CSV
results_df.to_csv('../data/model_comparison_results.csv', index=False)



/opt/anaconda3/envs/env_lmp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/env_lmp/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

                   Model          Outliers        Scaling     Feature Set  \
0    Logistic Regression     With Outliers     No Scaling    All Features   
1          Decision Tree     With Outliers     No Scaling    All Features   
2          Random Forest     With Outliers     No Scaling    All Features   
3                    SVM     With Outliers     No Scaling    All Features   
4                    KNN     With Outliers     No Scaling    All Features   
..                   ...               ...            ...             ...   
115  Logistic Regression  Without Outliers  Normalization  No Family Size   
116        Decision Tree  Without Outliers  Normalization  No Family Size   
117        Random Forest  Without Outliers  Normalization  No Family Size   
118                  SVM  Without Outliers  Normalization  No Family Size   
119                  KNN  Without Outliers  Normalization  No Family Size   

     Accuracy  
0    0.797619  
1    0.753720  
2    0.774554  
3    0.6808

In [22]:
#display all rows
pd.set_option('display.max_rows', None)

results_df.sort_values(by = "Accuracy", ascending = False)

,Model,Outliers,Scaling,Feature Set,Accuracy
83,SVM,Without Outliers,Standardization,All Features,0.815408
104,KNN,Without Outliers,Normalization,All Features,0.813120
84,KNN,Without Outliers,Standardization,All Features,0.807018
23,SVM,With Outliers,Standardization,All Features,0.806548
103,SVM,Without Outliers,Normalization,All Features,0.806255
100,Logistic Regression,Without Outliers,Normalization,All Features,0.806255
80,Logistic Regression,Without Outliers,Standardization,All Features,0.805492
60,Logistic Regression,Without Outliers,No Scaling,All Features,0.803966
40,Logistic Regression,With Outliers,Normalization,All Features,0.799851
43,SVM,With Outliers,Normalization,All Features,0.799107
